<a href="https://colab.research.google.com/github/nasstyalo/-25/blob/main/%D0%A2%D0%B5%D1%81%D1%82%D0%BE%D0%B2%D0%BE%D0%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz



--2024-10-17 19:36:01--  https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M   918KB/s    in 86s     

2024-10-17 19:37:28 (954 KB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [6]:
!tar -xvzf aclImdb_v1.tar.gz


Streaming output truncated to the last 5000 lines.
aclImdb/train/unsup/44983_0.txt
aclImdb/train/unsup/44982_0.txt
aclImdb/train/unsup/44981_0.txt
aclImdb/train/unsup/44980_0.txt
aclImdb/train/unsup/44979_0.txt
aclImdb/train/unsup/44978_0.txt
aclImdb/train/unsup/44977_0.txt
aclImdb/train/unsup/44976_0.txt
aclImdb/train/unsup/44975_0.txt
aclImdb/train/unsup/44974_0.txt
aclImdb/train/unsup/44973_0.txt
aclImdb/train/unsup/44972_0.txt
aclImdb/train/unsup/44971_0.txt
aclImdb/train/unsup/44970_0.txt
aclImdb/train/unsup/44969_0.txt
aclImdb/train/unsup/44968_0.txt
aclImdb/train/unsup/44967_0.txt
aclImdb/train/unsup/44966_0.txt
aclImdb/train/unsup/44965_0.txt
aclImdb/train/unsup/44964_0.txt
aclImdb/train/unsup/44963_0.txt
aclImdb/train/unsup/44962_0.txt
aclImdb/train/unsup/44961_0.txt
aclImdb/train/unsup/44960_0.txt
aclImdb/train/unsup/44959_0.txt
aclImdb/train/unsup/44958_0.txt
aclImdb/train/unsup/44957_0.txt
aclImdb/train/unsup/44956_0.txt
aclImdb/train/unsup/44955_0.txt
aclImdb/train/unsup/4

In [7]:
!ls aclImdb/train/pos | head


0_9.txt
10000_8.txt
10001_10.txt
10002_7.txt
10003_8.txt
10004_8.txt
10005_7.txt
10006_7.txt
10007_7.txt
10008_7.txt


In [8]:
import os

DATA_DIR = "aclImdb"  # Относительный путь в Colab

def load_data(data_dir):
    texts, labels = [], []
    for label_type in ['pos', 'neg']:
        dir_name = os.path.join(data_dir, label_type)
        if not os.path.exists(dir_name):
            raise FileNotFoundError(f"Directory not found: {dir_name}")
        for fname in os.listdir(dir_name):
            if fname.endswith('.txt'):
                with open(os.path.join(dir_name, fname), encoding='utf-8') as f:
                    texts.append(f.read())
                    labels.append(1 if label_type == 'pos' else 0)
    return texts, labels

# Загружаем данные
texts, labels = load_data(os.path.join(DATA_DIR, 'train'))
print(f"Loaded {len(texts)} reviews.")


Loaded 25000 reviews.


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# Разделение на тренировочные и тестовые данные
X_train, X_test, y_train, y_test = train_test_split(
    texts, labels, test_size=0.2, random_state=42
)

# Векторизация текстов с помощью TF-IDF
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Обучение модели
model = LogisticRegression(max_iter=100)
model.fit(X_train_tfidf, y_train)

# Предсказание на тестовых данных
y_pred = model.predict(X_test_tfidf)

# Оценка точности модели
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


Test Accuracy: 87.66%


In [11]:
import joblib

# Сохранение модели и векторизатора
joblib.dump(model, 'model.joblib')
joblib.dump(vectorizer, 'vectorizer.joblib')

print("Model and vectorizer saved.")


Model and vectorizer saved.


In [12]:
# Загрузка модели и векторизатора
model = joblib.load('model.joblib')
vectorizer = joblib.load('vectorizer.joblib')

# Пример отзыва для тестирования
sample_review = ["This movie was absolutely fantastic! The plot was engaging."]
sample_tfidf = vectorizer.transform(sample_review)

# Предсказание
prediction = model.predict(sample_tfidf)[0]
sentiment = 'Positive' if prediction == 1 else 'Negative'

print(f"Sentiment: {sentiment}")


Sentiment: Positive


In [23]:
!pip install django joblib pyngrok



In [29]:
!rm -rf movie_review_service


In [30]:
!django-admin startproject movie_review_service


In [53]:
%cd /content/movie_review_service/review_classifier


/content/movie_review_service/review_classifier


In [60]:
!mkdir templates
with open("templates/review_form.html", "w") as f:
    f.write('''
<!DOCTYPE html>
<html>
<head>
    <title>Movie Review</title>
</head>
<body>
    <h1>Enter your movie review</h1>
    <form method="post">
        {% csrf_token %}
        {{ form.as_p }}
        <button type="submit">Submit</button>
    </form>
</body>
</html>
''')
with open("templates/result.html", "w") as f:
    f.write('''
<!DOCTYPE html>
<html>
<head>
    <title>Review Result</title>
</head>
<body>
    <h1>Review Result</h1>
    <p>Rating: {{ rating }}</p>
    <p>Sentiment: {{ sentiment }}</p>
    <a href="/">Submit another review</a>
</body>
</html>
''')
with open("/content/movie_review_service/review_classifier/urls.py", "w") as f:
    f.write('''
from django.urls import path
from . import views

urlpatterns = [
    path('', views.classify_review, name='classify_review'),
]
''')
with open("/content/movie_review_service/movie_review_service/urls.py", "w") as f:
    f.write('''
from django.contrib import admin
from django.urls import path, include

urlpatterns = [
    path('admin/', admin.site.urls),
    path('', include('review_classifier.urls')),
]
''')
with open("/content/movie_review_service/review_classifier/forms.py", "w") as f:
    f.write('''
from django import forms

class ReviewForm(forms.Form):
    review = forms.CharField(
        label='Your Review',
        widget=forms.Textarea(attrs={'rows': 5, 'cols': 40})
    )
''')
with open("/content/movie_review_service/review_classifier/views.py", "w") as f:
    f.write('''
from django.shortcuts import render
from .forms import ReviewForm
import joblib

# Загрузка модели и векторизатора
model = joblib.load('/content/model.joblib')
vectorizer = joblib.load('/content/vectorizer.joblib')

def predict_review(text):
    transformed_text = vectorizer.transform([text])
    prediction = model.predict(transformed_text)[0]
    rating = 10 if prediction == 1 else 1
    sentiment = 'Positive' if prediction == 1 else 'Negative'
    return rating, sentiment

def classify_review(request):
    if request.method == 'POST':
        form = ReviewForm(request.POST)
        if form.is_valid():
            review_text = form.cleaned_data['review']
            rating, sentiment = predict_review(review_text)
            return render(request, 'result.html', {'rating': rating, 'sentiment': sentiment})
    else:
        form = ReviewForm()
    return render(request, 'review_form.html', {'form': form})
''')


mkdir: cannot create directory ‘templates’: File exists


In [73]:
!ls -R /content/movie_review_service
!cat /content/movie_review_service/movie_review_service/settings.py


/content/movie_review_service:
manage.py  movie_review_service  review_classifier

/content/movie_review_service/movie_review_service:
asgi.py  __init__.py  __pycache__  settings.py	urls.py  wsgi.py

/content/movie_review_service/movie_review_service/__pycache__:
__init__.cpython-310.pyc  settings.cpython-310.pyc  urls.cpython-310.pyc

/content/movie_review_service/review_classifier:
admin.py  forms.py     migrations  templates  urls.py
apps.py   __init__.py  models.py   tests.py   views.py

/content/movie_review_service/review_classifier/migrations:
__init__.py

/content/movie_review_service/review_classifier/templates:
result.html  review_form.html
"""
Django settings for movie_review_service project.

Generated by 'django-admin startproject' using Django 5.1.2.

For more information on this file, see
https://docs.djangoproject.com/en/5.1/topics/settings/

For the full list of settings and their values, see
https://docs.djangoproject.com/en/5.1/ref/settings/
"""

from pathlib import 

In [77]:
# Django settings for movie_review_service project.

import os
from pathlib import Path

# Определение BASE_DIR без использования __file__
BASE_DIR = Path(os.getcwd()).resolve()

SECRET_KEY = 'django-insecure-*!ef9*h^22wlu2lr^!z#bco9c9^p9g+7(=wdt*_asduq!1sf6_'

DEBUG = True

ALLOWED_HOSTS = []

INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'review_classifier',
]

MIDDLEWARE = [
    'django.middleware.security.SecurityMiddleware',
    'django.contrib.sessions.middleware.SessionMiddleware',
    'django.middleware.common.CommonMiddleware',
    'django.middleware.csrf.CsrfViewMiddleware',
    'django.contrib.auth.middleware.AuthenticationMiddleware',
    'django.contrib.messages.middleware.MessageMiddleware',
    'django.middleware.clickjacking.XFrameOptionsMiddleware',
]

ROOT_URLCONF = 'movie_review_service.urls'

TEMPLATES = [
    {
        'BACKEND': 'django.template.backends.django.DjangoTemplates',
        'DIRS': [],
        'APP_DIRS': True,
        'OPTIONS': {
            'context_processors': [
                'django.template.context_processors.debug',
                'django.template.context_processors.request',
                'django.contrib.auth.context_processors.auth',
                'django.contrib.messages.context_processors.messages',
            ],
        },
    },
]

WSGI_APPLICATION = 'movie_review_service.wsgi.application'

DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.sqlite3',
        'NAME': BASE_DIR / 'db.sqlite3',
    }
}

AUTH_PASSWORD_VALIDATORS = [
    {
        'NAME': 'django.contrib.auth.password_validation.UserAttributeSimilarityValidator',
    },
    {
        'NAME': 'django.contrib.auth.password_validation.MinimumLengthValidator',
    },
    {
        'NAME': 'django.contrib.auth.password_validation.CommonPasswordValidator',
    },
    {
        'NAME': 'django.contrib.auth.password_validation.NumericPasswordValidator',
    },
]

LANGUAGE_CODE = 'en-us'

TIME_ZONE = 'UTC'

USE_I18N = True

USE_TZ = True

STATIC_URL = 'static/'

DEFAULT_AUTO_FIELD = 'django.db.models.BigAutoField'


In [78]:
with open('/content/movie_review_service/movie_review_service/settings.py', 'w') as f:
    f.write('''
import os
from pathlib import Path

BASE_DIR = Path(os.getcwd()).resolve()

SECRET_KEY = 'django-insecure-*!ef9*h^22wlu2lr^!z#bco9c9^p9g+7(=wdt*_asduq!1sf6_'

DEBUG = True

ALLOWED_HOSTS = []

INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'review_classifier',
]

MIDDLEWARE = [
    'django.middleware.security.SecurityMiddleware',
    'django.contrib.sessions.middleware.SessionMiddleware',
    'django.middleware.common.CommonMiddleware',
    'django.middleware.csrf.CsrfViewMiddleware',
    'django.contrib.auth.middleware.AuthenticationMiddleware',
    'django.contrib.messages.middleware.MessageMiddleware',
    'django.middleware.clickjacking.XFrameOptionsMiddleware',
]

ROOT_URLCONF = 'movie_review_service.urls'

TEMPLATES = [
    {
        'BACKEND': 'django.template.backends.django.DjangoTemplates',
        'DIRS': [],
        'APP_DIRS': True,
        'OPTIONS': {
            'context_processors': [
                'django.template.context_processors.debug',
                'django.template.context_processors.request',
                'django.contrib.auth.context_processors.auth',
                'django.contrib.messages.context_processors.messages',
            ],
        },
    },
]

WSGI_APPLICATION = 'movie_review_service.wsgi.application'

DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.sqlite3',
        'NAME': BASE_DIR / 'db.sqlite3',
    }
}

AUTH_PASSWORD_VALIDATORS = [
    {
        'NAME': 'django.contrib.auth.password_validation.UserAttributeSimilarityValidator',
    },
    {
        'NAME': 'django.contrib.auth.password_validation.MinimumLengthValidator',
    },
    {
        'NAME': 'django.contrib.auth.password_validation.CommonPasswordValidator',
    },
    {
        'NAME': 'django.contrib.auth.password_validation.NumericPasswordValidator',
    },
]

LANGUAGE_CODE = 'en-us'

TIME_ZONE = 'UTC'

USE_I18N = True

USE_TZ = True

STATIC_URL = 'static/'

DEFAULT_AUTO_FIELD = 'django.db.models.BigAutoField'
''')


In [79]:
!python /content/movie_review_service/manage.py migrate
!python /content/movie_review_service/manage.py runserver 0.0.0.0:8000


Operations to perform:
  Apply all migrations: admin, auth, contenttypes, sessions
Running migrations:
  Applying contenttypes.0001_initial... OK
  Applying auth.0001_initial... OK
  Applying admin.0001_initial... OK
  Applying admin.0002_logentry_remove_auto_add... OK
  Applying admin.0003_logentry_add_action_flag_choices... OK
  Applying contenttypes.0002_remove_content_type_name... OK
  Applying auth.0002_alter_permission_name_max_length... OK
  Applying auth.0003_alter_user_email_max_length... OK
  Applying auth.0004_alter_user_username_opts... OK
  Applying auth.0005_alter_user_last_login_null... OK
  Applying auth.0006_require_contenttypes_0002... OK
  Applying auth.0007_alter_validators_add_error_messages... OK
  Applying auth.0008_alter_user_username_max_length... OK
  Applying auth.0009_alter_user_last_name_max_length... OK
  Applying auth.0010_alter_group_name_max_length... OK
  Applying auth.0011_update_proxy_permissions... OK
  Applying auth.0012_alter_user_first_name_max_l

In [82]:
!apt-get install -y nodejs npm
!npm install -g localtunnel




Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  gyp javascript-common libc-ares2 libjs-events libjs-highlight.js libjs-inherits
  libjs-is-typedarray libjs-psl libjs-source-map libjs-sprintf-js libjs-typedarray-to-buffer
  libnode-dev libnode72 libnotify-bin libnotify4 libuv1-dev node-abab node-abbrev node-agent-base
  node-ansi-regex node-ansi-styles node-ansistyles node-aproba node-archy node-are-we-there-yet
  node-argparse node-arrify node-asap node-asynckit node-balanced-match node-brace-expansion
  node-builtins node-cacache node-chalk node-chownr node-clean-yaml-object node-cli-table
  node-clone node-color-convert node-color-name node-colors node-columnify node-combined-stream
  node-commander node-console-control-strings node-copy-concurrently node-core-util-is
  node-coveralls node-cssom node-cssstyle node-debug node-decompress-response node-defaults
  node-delayed-stream n

In [85]:
!python /content/movie_review_service/manage.py runserver 0.0.0.0:8000


Watching for file changes with StatReloader
Performing system checks...

System check identified no issues (0 silenced).
October 17, 2024 - 20:50:37
Django version 5.1.2, using settings 'movie_review_service.settings'
Starting development server at http://0.0.0.0:8000/
Quit the server with CONTROL-C.



In [89]:
!lt --port 8000



your url is: https://loose-doodles-brush.loca.lt
^C
